In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
sns.set_style('white')
sns.set_context('talk')
import numpy as np
import pandas as pd
import collections
import addict

In [4]:
name = 'error-vs-time-importance-sampling'

In [5]:
hvm_dir = pathlib.Path('/Users/jaan/log/transfer_tigergpu/2019-06-23/error-vs-time/')
hvm_dir = pathlib.Path('/Users/jaan/log/transfer_tigergpu/2019-07-06/importance-sampling/')
log_dict = addict.Dict()
for f in hvm_dir.rglob('*.csv'):
  name = str(f.parts[-2])
  split = name.split('_')
  system_size = int(name.lstrip('system_size=')[:2])
  beta = float(name.split('beta=')[1])
  log_dict['hvm'][system_size][beta] = f

In [6]:
van_dir = pathlib.Path('/Users/jaan/log/transfer_tigergpu/2019-06-23/test/stat-mech-van')
for f in van_dir.rglob('*.csv'):
  name = f.parts[-3]
  if 'periodic' in name:
    system_size = int(name.split('L')[1][:2])
    beta = float(name.split('beta')[1])
    log_dict['van'][system_size][beta] = f

In [7]:
log_dict = log_dict.to_dict()

In [8]:
df_dict = {}
for method in ['van', 'hvm']:
  df_dict[method] = {}
  for system_size in log_dict[method].keys():
    df_dict[method][system_size] = {}
    for beta in log_dict[method][system_size].keys():
#      if beta != 1.0:
      df = pd.read_csv(
        log_dict[method][system_size][beta],
        header=None,
        usecols=range(2),
        names=['step', 'free_energy'])
      if method == 'van':
        df.step = df.step.apply(lambda x: int(x.split('=')[1]))
        df.free_energy = df.free_energy.apply(lambda x: float(x.split('=')[1]))
      # convert using 6h runtime to timepoints
      df['time'] = df.step * (6 * 60 * 60 / df.step.values[-1])
      df_dict[method][system_size][beta] = df

In [9]:
labels = {'van': 'VAN', 'hvm': 'HVM'}
linestyle = {'van': '--', 'hvm': '-'}
palettes = {'van': sns.light_palette('orange', reverse=True, n_colors=4),
            'hvm': sns.light_palette('blue', reverse=True, n_colors=4)}
sizes_to_plot = [16, 32, 64]
for beta in df_dict['hvm'][16].keys():
  fig, ax = plt.subplots(figsize=(5 * 1.618, 5))
  for method in ['hvm', 'van']:
    for i, system_size in enumerate(sizes_to_plot):
      df = df_dict[method][system_size][beta]
      label = f'{labels[method]} L={system_size}'
      ax.plot(df.time.values / 60 / 60,
                  df.free_energy.values,
                  label=label,
                  color=palettes[method][i],
                  linestyle=linestyle[method],
      )
  ax.legend()
  ax.set_ylim([-2.2, -0.5])
  ax.set_xlim([0, 6])
  ax.set(xlabel='Time (hours)', ylabel='Free energy')
  ax.yaxis.grid()
  sns.despine(right=True, left=True)
  plt.savefig(f'../../doc/notes/hierarchical-elbo/fig/{name}_beta={beta}.pdf', bbox_inches='tight')
  plt.close()


In [1]:
'=10'.rstrip('10')

'='

In [ ]:
d